In [1]:
import os 

In [3]:
%pwd

'c:\\Users\\46236358\\OneDrive - GLOBAL HITSS\\Documentos\\git personal\\end-to-end-ML-project-whit-MLflow\\research'

In [4]:
os.chdir('../')

In [ ]:
# Es necesario estar en la "raiz" del proyecto
%pwd

'c:\\Users\\46236358\\OneDrive - GLOBAL HITSS\\Documentos\\git personal\\end-to-end-ML-project-whit-MLflow'

In [ ]:
# 3. Update the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Despues de esto se va a constants/__init__.py y se crean las rutas ahi


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
# 5.Update the configuration manager in src config
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        SCHEMA_FILE_PATH = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

    

In [9]:
import os 
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [15]:
# 6. Update the components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extrae el archivo zip en la directorio 
        La funcion devuelve None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [16]:
# 7. Update the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise(e)

[2025-12-18 17:22:38,227: INFO: common]: yaml file: config\config.yaml loaded successfully]
[2025-12-18 17:22:38,230: INFO: common]: yaml file: params.yaml loaded successfully]
[2025-12-18 17:22:38,234: INFO: common]: yaml file: schema.yaml loaded successfully]
[2025-12-18 17:22:38,236: INFO: common]: created directory at: artifacts]
[2025-12-18 17:22:38,238: INFO: common]: created directory at: artifacts/data_ingestion]
[2025-12-18 17:22:39,016: INFO: 2627750695]: artifacts\data_ingestion\data.zip downloaded! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9A3E:261528:92C41:B20F5:69447EA7
Accept-Ranges: bytes
Date: T